
# Loading and Experimenting with a Pre-trained Large Language Model (LLM)

In this exercise, we will load a pre-trained Large Language Model (LLM) from Hugging Face and experiment with it to generate text. We'll use GPT-2, a popular LLM, and explore how to generate different text outputs based on user prompts.

### Steps:
1. Install the necessary libraries.
2. Load the GPT-2 model from Hugging Face.
3. Generate text based on user input.
4. Experiment with different prompts and parameters (such as length and temperature) to see how they affect the generated text.
    

In [ ]:

# Install the required libraries
!pip install transformers torch
    


## Step 1: Load the GPT-2 Model and Tokenizer

We'll load the GPT-2 model and tokenizer from the Hugging Face Transformers library.
    

In [ ]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Move the model to GPU if available
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
    


## Step 2: Text Generation with GPT-2

Now, we'll create a function that takes a user input as a prompt and generates text using the GPT-2 model. You can modify the prompt and parameters like `max_length` and `temperature` to experiment with different outputs.
The `generate_text` function takes the following inputs:
- `prompt`: A string input that serves as the starting point for text generation.
- `max_length`: The maximum length of the generated text (default is 50).
- `temperature`: A parameter that controls the randomness or creativity of the generated text (default is 1.0, higher values result in more randomness).
- `num_return_sequences=1`: Specifies that only one sequence of text should be generated.
- `no_repeat_ngram_size=2`: Ensures that no 2-grams (two consecutive words) are repeated in the generated text.
- `do_sample=True`: Enables sampling, allowing the model to explore different possible continuations for the text.


After the text is generated, the function decodes the tokenized output back into human-readable text.

```python
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
```

- `tokenizer.decode(outputs[0])`: Converts the tokenized output back into a string.
- `skip_special_tokens=True`: Removes any special tokens that the model might have added during generation (such as end-of-sequence tokens).

Finally, the decoded text is returned as the output of the function.
    

In [3]:

def generate_text(prompt, max_length=50, temperature=1.0):
    # Tokenize the input prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Generate text using the model
    outputs = model.generate(
        inputs, 
        max_length=max_length, 
        temperature=temperature, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2,
        do_sample=True
    )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text
    


## Step 3: Experiment with Different Prompts

We will now create an interactive Gradio interface where you can input your own prompts and generate text using the GPT-2 model. You can also adjust the `max_length` (number of tokens generated) and `temperature` (controls the randomness of the output).
    

In [ ]:

import gradio as gr

# Create the Gradio interface
def generate_interface(prompt, max_length, temperature):
    return generate_text(prompt, max_length=max_length, temperature=temperature)

interface = gr.Interface(
    fn=generate_interface,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your prompt here", label="Prompt"),
        gr.Slider(10, 100, value=50, label="Max Length"),
        gr.Slider(0.7, 1.3, value=1.0, label="Temperature")
    ],
    outputs="text",
    title="Text Generation with GPT-2",
    description="Enter a prompt and experiment with different parameters to generate text."
)

interface.launch()
    